<a href="https://colab.research.google.com/github/orilevi2809/DL/blob/main/project/mobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from PIL import Image
import pickle
import numpy as np

from google.colab import drive
import timm
import torch.nn as nn
!pip install timm

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
IMG_SIZE = 224 #TODO: check if we can minize the size of the image e.g. 32*32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img_dir = "./drive/MyDrive/labels/"
labels_dict = {"crimp": 0, "pinch": 1, "pocket": 2, "jug": 3, "edge": 4, "sloper": 5}

# Define the transform for image preprocessing
no_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

no_transform_norm = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform_norm = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transform2_norm = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [51]:
def save_model(model, model_file):
    # Save the model's state dictionary to disk
    torch.save(model.state_dict(), model_file)


In [52]:
def load_images_paths(img_dir, labels):
    data = []
    target = []
    label_count = {}  # Dictionary to store the count of images for each label
    for label in labels:
        label_dir = os.path.join(img_dir, label)
        count = 0  # Initialize the count for the current label
        for filename in os.listdir(label_dir):
            if filename.endswith(".jpg") or filename.endswith(".jpeg")or filename.endswith(".png"):
                img_path = os.path.join(label_dir, filename)
                data.append(img_path)
                target.append(labels_dict[label])
                count += 1  # Increment the count for the current label
        label_count[label] = count  # Store the count in the dictionary

    # Print the count of images for each label
    for label, count in label_count.items():
        print(f"Number of images for label '{label}': {count}")
    return data, target , label_count


In [53]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    if isinstance(img, torch.Tensor):
        img = img.numpy()
        img = np.transpose(img, (1, 2, 0))
        img = (img * 255).astype(np.uint8)
        plt.imshow(img)
    else:  # Assuming img is a PIL.Image object
        plt.imshow(img)

    plt.show()


In [54]:
def load_image_data(img_paths, labels, transform,label_counts, augment_unbalanced=False):
    if augment_unbalanced:
        is_unbalanced = any(count < max(label_counts.values()) for count in label_counts.values())
        print(label_counts)
        inv_labels_dict = {0:"crimp",1: "pinch",2: "pocket",3: "jug",4: "edge",5: "sloper"}

        if is_unbalanced:
            print("unbalanced")
            augmented_data = []
            augmented_target = []
            #TODO: add while all equal
            for img_path, label in zip(img_paths, labels):
                img = Image.open(img_path).convert("RGB")
                augmented_data.append(no_transform_norm(img))
                augmented_target.append(label)
                if label_counts[label] < max(label_counts.values()):
                    augmented_img = train_transform_norm(img)
                    #imshow(augmented_img)
                    augmented_data.append(augmented_img)
                    augmented_target.append(label)
                    label_counts[label] += 1
                if label_counts[label] < max(label_counts.values()):
                    augmented_img = train_transform2_norm(img)
                    #imshow(img)
                    #imshow(augmented_img)
                    augmented_data.append(augmented_img)
                    augmented_target.append(label)
                    label_counts[label] += 1
            print(label_counts)
            return augmented_data, augmented_target
        else:
              data = []
              target = []
              label_counts = {label: 0 for label in set(labels)}
              for img_path, label in zip(img_paths, labels):
                  img = Image.open(img_path).convert("RGB")
                  img = no_transform_norm(img)
                  data.append(img)
                  target.append(label)
                  label_counts[label] += 1
              return data, target
    else:
        if isinstance(img_paths[0], torch.Tensor):  # Check if data is already in tensor format
            return img_paths, labels
        else:
            data = []
            target = []
            for img_path, label in zip(img_paths, labels):
                img = Image.open(img_path).convert("RGB")
                img = no_transform_norm(img)
                data.append(img)
                target.append(label)
            return data, target

In [55]:

def train_classifier(train_dataloader, val_dataloader, model, criterion, optimizer, device,epochs):
    for epoch in range(epochs):  # 10 is the best!!
        running_loss = 0.0
        total_correct = 0
        total_samples = 0

        # Switch model to training mode
        model.train()

        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate the number of correct predictions in the current batch
            _, predicted = torch.max(outputs.data, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        train_accuracy = 100 * total_correct / total_samples

        # Switch model to evaluation mode for validation
        model.eval()

        val_correct = 0
        val_samples = 0
        val_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(val_dataloader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_correct += (predicted == labels).sum().item()
                val_samples += labels.size(0)

        val_accuracy = 100 * val_correct / val_samples

        print(f'Epoch: {epoch + 1}, Training loss: {running_loss / len(train_dataloader)}, Training accuracy: {train_accuracy}%, Validation loss: {val_loss / len(val_dataloader)}, Validation accuracy: {val_accuracy}%')

    print('Finished Training')
    return model

In [56]:
def evaluate_classifier(test_dataloader, model, device):
    model.eval()  # Switch model to evaluation mode
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Move tensors to CPU before converting to numpy arrays
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy}%')
    # Generate classification report and confusion matrix
    report = classification_report(all_labels, all_predictions, zero_division=1)
    matrix = confusion_matrix(all_labels, all_predictions)
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(matrix)


In [160]:
img_paths, labels,_ = load_images_paths(img_dir, labels_dict.keys())
print("total data= ", len(img_paths))

# Split the data into train, validation, and test sets (e.g., 70%, 15%, 15%)
img_paths_train, img_paths_test, labels_train, labels_test = train_test_split(img_paths, labels, test_size=0.3,stratify=labels, random_state=42)
img_paths_val, img_paths_test, labels_val, labels_test = train_test_split(img_paths_test, labels_test,test_size=0.5, stratify=labels_test,random_state=42)


label_counts = {label: 0 for label in set(labels)}
for label in labels_train:
  label_counts[label] += 1

data_train, labels_train = load_image_data(img_paths_train, labels_train, train_transform,label_counts,augment_unbalanced=True)
data_val, labels_val = load_image_data(img_paths_val, labels_val, no_transform,label_counts,augment_unbalanced=False)
data_test, labels_test = load_image_data(img_paths_test, labels_test, no_transform,label_counts,augment_unbalanced=False)


# Convert to tensors
data_train = torch.stack(data_train)
labels_train = torch.tensor(labels_train)

data_val = torch.stack(data_val)
labels_val = torch.tensor(labels_val)

data_test = torch.stack(data_test)
labels_test = torch.tensor(labels_test)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(data_train, labels_train)
val_dataset = TensorDataset(data_val, labels_val)
test_dataset = TensorDataset(data_test, labels_test)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)



Number of images for label 'crimp': 161
Number of images for label 'pinch': 485
Number of images for label 'pocket': 178
Number of images for label 'jug': 101
Number of images for label 'edge': 113
Number of images for label 'sloper': 262
total data=  1300
{0: 113, 1: 339, 2: 125, 3: 71, 4: 79, 5: 183}
unbalanced
{0: 339, 1: 339, 2: 339, 3: 213, 4: 237, 5: 339}


In [ ]:
# Save the data loaders
pickle_dir = "./drive/MyDrive/pickles/"

with open(pickle_dir+'train_dataloader.pkl', 'wb') as f:
    pickle.dump(train_dataloader, f)

with open(pickle_dir+'val_dataloader.pkl', 'wb') as f:
    pickle.dump(val_dataloader, f)

with open(pickle_dir+'test_dataloader.pkl', 'wb') as f:
    pickle.dump(test_dataloader, f)

In [ ]:
# Load the data loaders
pickle_dir = "./drive/MyDrive/pickles/"
with open(pickle_dir+'train_dataloader.pkl', 'rb') as f:
    train_dataloader = pickle.load(f)

with open(pickle_dir+'val_dataloader.pkl', 'rb') as f:
    val_dataloader = pickle.load(f)

with open(pickle_dir+'test_dataloader.pkl', 'rb') as f:
    test_dataloader = pickle.load(f)

In [233]:
def get_model_mobileNet():
  # Load pre-trained MobileNetV2 model and replace the classifier layer
  model = models.mobilenet_v2(pretrained=True)
  #for param in model.parameters():
  #    param.requires_grad = False

  # Enable gradient computation for all parameters in the classifier layers
  #for param in model.classifier.parameters():
  #    param.requires_grad = True

  # Only train the last three layers
  #for param in model.classifier[-3:].parameters():
  #    param.requires_grad = True

  model.classifier[1] = nn.Linear(model.last_channel, len(labels_dict))
  model = model.to(device)
  return model



In [244]:

def get_model_efficientNet():
    # Load pre-trained EfficientNet model and replace the classifier layer
    model = timm.create_model('efficientnet_b2', pretrained=True)

    # Freeze all parameters
    #model.requires_grad_(True)

    # Unfreeze the last three blocks
    # for param in model.blocks[-3:].parameters():
    #    param.requires_grad = True

    num_features = model.classifier.in_features
    model.classifier = nn.Linear(num_features, len(labels_dict))

    model = model.to(device)
    return model

In [194]:
def get_model_VGG():
    model = models.vgg19(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    # Unfreeze the last three layers
    for param in model.features[-3:].parameters():
        param.requires_grad = True
    #print(len(model.parameters() == True))
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, len(labels_dict))  # assuming labels_dict is defined elsewhere

    model = model.to(device)
    return model


In [220]:

def get_model_densenet():
    # Load pre-trained DenseNet model and replace the classifier layer
    model = models.densenet201(pretrained=True)

    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the last layer block (you may need to adjust this part to suit your needs)
    for param in model.features.denseblock4.parameters():
        param.requires_grad = True
    
    

    num_features = model.classifier.in_features
    model.classifier = nn.Linear(num_features, len(labels_dict))  # assuming labels_dict is defined elsewhere

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return model

In [206]:

def get_model_resnet():
    # Load pre-trained ResNet model and replace the classifier layer
    model = models.resnet152(pretrained=True)

    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the last layer
    for param in model.layer4.parameters():
        param.requires_grad = True

    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, len(labels_dict))  # assuming labels_dict is defined elsewhere

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return model

In [140]:

def get_model_alexnet():
    # Load pre-trained AlexNet model and replace the classifier layer
    model = models.alexnet(pretrained=True)

    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the last layer
    for param in model.features[-4:].parameters():
        param.requires_grad = True

    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, len(labels_dict))  # assuming labels_dict is defined elsewhere

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return model

In [227]:
def train_model(model,epochs):
  # Define loss criterion and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

  model = train_classifier(train_dataloader, val_dataloader, model, criterion, optimizer, device,epochs)

  if not os.path.exists('hold_classifier_MobileNet.pth'):
      # Save the MobileNetV2 model
      model_file = 'hold_classifier_MobileNet.pth'
      save_model(model, model_file)

      # Load the model for inference
      model.load_state_dict(torch.load('hold_classifier_MobileNet.pth'))
  return model



In [64]:

if not os.path.exists('hold_classifier_MobileNet.pth'):
    # Save the MobileNetV2 model
    model_file = 'hold_classifier_MobileNet.pth'
    save_model(model, model_file)

    # Load the model for inference
    model.load_state_dict(torch.load('hold_classifier_MobileNet.pth'))




In [234]:
model = get_model_mobileNet()
trained_model = train_model(model,epochs=3)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.068822375659285, Training accuracy: 60.35437430786268%, Validation loss: 2.6527495980262756, Validation accuracy: 52.82051282051282%
Epoch: 2, Training loss: 0.6376561378610546, Training accuracy: 78.18383167220377%, Validation loss: 1.2410312741994858, Validation accuracy: 63.58974358974359%
Epoch: 3, Training loss: 0.5108591081767246, Training accuracy: 81.28460686600222%, Validation loss: 1.6254142671823502, Validation accuracy: 66.15384615384616%
Finished Training
Accuracy of the model on the test images: 66.15384615384616%
Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.46      0.51        24
           1       0.72      0.73      0.72        73
           2       1.00      0.85      0.92        27
           3       0.38      0.53      0.44        15
           4       0.48      0.59      0.53        17
           5       0.65      0.62      0.63        39

    accuracy                        

In [169]:
model5 = get_model_mobileNet()
trained_model5 = train_model(model5,epochs=5)
evaluate_classifier(test_dataloader, trained_model5, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.1033947241717372, Training accuracy: 58.91472868217054%, Validation loss: 1.4738274812698364, Validation accuracy: 64.61538461538461%
Epoch: 2, Training loss: 0.5990191677521015, Training accuracy: 78.3499446290144%, Validation loss: 1.9151490926742554, Validation accuracy: 66.66666666666667%
Epoch: 3, Training loss: 0.4040071614857378, Training accuracy: 86.2126245847176%, Validation loss: 2.091602236032486, Validation accuracy: 62.56410256410256%
Epoch: 4, Training loss: 0.33475140148195726, Training accuracy: 88.03986710963456%, Validation loss: 1.3928267359733582, Validation accuracy: 66.15384615384616%
Epoch: 5, Training loss: 0.26023830858797864, Training accuracy: 90.91915836101883%, Validation loss: 1.7810733616352081, Validation accuracy: 68.2051282051282%
Finished Training
Accuracy of the model on the test images: 68.71794871794872%
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.58      0

In [170]:
model = get_model_mobileNet()
trained_model = train_model(model,epochs=10)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.0522292174141983, Training accuracy: 60.79734219269103%, Validation loss: 1.7706602811813354, Validation accuracy: 62.56410256410256%
Epoch: 2, Training loss: 0.6076736727665211, Training accuracy: 78.01771871539313%, Validation loss: 1.5123671740293503, Validation accuracy: 66.66666666666667%
Epoch: 3, Training loss: 0.36622859126534957, Training accuracy: 85.76965669988925%, Validation loss: 1.3979102671146393, Validation accuracy: 65.64102564102564%
Epoch: 4, Training loss: 0.3288089853936228, Training accuracy: 87.98449612403101%, Validation loss: 2.29059299826622, Validation accuracy: 66.15384615384616%
Epoch: 5, Training loss: 0.2514229725147116, Training accuracy: 90.91915836101883%, Validation loss: 2.408460944890976, Validation accuracy: 64.1025641025641%
Epoch: 6, Training loss: 0.1386596741604394, Training accuracy: 95.34883720930233%, Validation loss: 2.176498234272003, Validation accuracy: 67.6923076923077%
Epoch: 7, Training loss: 0.094209488352

In [173]:
model = get_model_mobileNet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.0350793908382285, Training accuracy: 61.40642303433001%, Validation loss: 1.5941693484783173, Validation accuracy: 56.41025641025641%
Epoch: 2, Training loss: 0.5694051333542528, Training accuracy: 79.67884828349945%, Validation loss: 1.457981362938881, Validation accuracy: 61.53846153846154%
Epoch: 3, Training loss: 0.4081995548873112, Training accuracy: 85.27131782945736%, Validation loss: 1.5894124507904053, Validation accuracy: 57.43589743589744%
Epoch: 4, Training loss: 0.2656496886549325, Training accuracy: 90.58693244739756%, Validation loss: 2.1410719007253647, Validation accuracy: 63.58974358974359%
Epoch: 5, Training loss: 0.221304417941077, Training accuracy: 92.19269102990033%, Validation loss: 1.494978904724121, Validation accuracy: 60.0%
Epoch: 6, Training loss: 0.22078728547384, Training accuracy: 92.41417497231451%, Validation loss: 2.253840982913971, Validation accuracy: 64.61538461538461%
Epoch: 7, Training loss: 0.1555251512548019, Training

In [247]:
model = get_model_efficientNet()
trained_model = train_model(model,epochs=3)
evaluate_classifier(test_dataloader, trained_model, device)

Epoch: 1, Training loss: 0.9346702592126255, Training accuracy: 66.27906976744185%, Validation loss: 1.0351901203393936, Validation accuracy: 68.71794871794872%
Epoch: 2, Training loss: 0.3525650866072753, Training accuracy: 89.14728682170542%, Validation loss: 1.2667529731988907, Validation accuracy: 68.2051282051282%
Epoch: 3, Training loss: 0.16776757610255275, Training accuracy: 94.18604651162791%, Validation loss: 1.9621455073356628, Validation accuracy: 64.1025641025641%
Finished Training
Accuracy of the model on the test images: 65.64102564102564%
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.38      0.46        24
           1       0.84      0.66      0.74        73
           2       0.91      0.78      0.84        27
           3       0.36      0.27      0.31        15
           4       0.42      0.65      0.51        17
           5       0.56      0.90      0.69        39

    accuracy                        

In [248]:
model = get_model_efficientNet()
trained_model = train_model(model,epochs=5)
evaluate_classifier(test_dataloader, trained_model, device)

Epoch: 1, Training loss: 0.9273959346886339, Training accuracy: 67.66334440753046%, Validation loss: 1.1647311002016068, Validation accuracy: 72.3076923076923%
Epoch: 2, Training loss: 0.31001768697952403, Training accuracy: 90.36544850498339%, Validation loss: 1.5290029048919678, Validation accuracy: 69.74358974358974%
Epoch: 3, Training loss: 0.17499440548748807, Training accuracy: 94.46290143964562%, Validation loss: 1.4938266277313232, Validation accuracy: 70.25641025641026%
Epoch: 4, Training loss: 0.144726956905476, Training accuracy: 96.29014396456257%, Validation loss: 1.046379268169403, Validation accuracy: 72.82051282051282%
Epoch: 5, Training loss: 0.17797577805046377, Training accuracy: 94.73975636766335%, Validation loss: 1.8636125028133392, Validation accuracy: 71.7948717948718%
Finished Training
Accuracy of the model on the test images: 68.2051282051282%
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.46      0

In [249]:
model = get_model_efficientNet()
trained_model = train_model(model,epochs=10)
evaluate_classifier(test_dataloader, trained_model, device)

Epoch: 1, Training loss: 0.9321486960197317, Training accuracy: 66.00221483942414%, Validation loss: 0.9367904663085938, Validation accuracy: 74.35897435897436%
Epoch: 2, Training loss: 0.29147311068814374, Training accuracy: 91.25138427464009%, Validation loss: 2.637096017599106, Validation accuracy: 65.64102564102564%
Epoch: 3, Training loss: 0.19412270920543834, Training accuracy: 93.57696566998892%, Validation loss: 1.2983133047819138, Validation accuracy: 70.25641025641026%
Epoch: 4, Training loss: 0.13694805330757437, Training accuracy: 95.73643410852713%, Validation loss: 2.2609579265117645, Validation accuracy: 63.07692307692308%
Epoch: 5, Training loss: 0.12959158735285545, Training accuracy: 95.9579180509413%, Validation loss: 1.5773551166057587, Validation accuracy: 68.2051282051282%
Epoch: 6, Training loss: 0.06298544196861572, Training accuracy: 98.28349944629015%, Validation loss: 2.1603429317474365, Validation accuracy: 71.28205128205128%
Epoch: 7, Training loss: 0.11231

In [250]:
model = get_model_efficientNet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

Epoch: 1, Training loss: 0.872700945056718, Training accuracy: 68.6046511627907%, Validation loss: 0.9906402677297592, Validation accuracy: 73.84615384615384%
Epoch: 2, Training loss: 0.28199248298488816, Training accuracy: 91.5282392026578%, Validation loss: 1.5252069532871246, Validation accuracy: 67.17948717948718%
Epoch: 3, Training loss: 0.2294543370090682, Training accuracy: 92.24806201550388%, Validation loss: 1.9571730196475983, Validation accuracy: 65.12820512820512%
Epoch: 4, Training loss: 0.16999169683148121, Training accuracy: 94.73975636766335%, Validation loss: 1.313079059123993, Validation accuracy: 64.61538461538461%
Epoch: 5, Training loss: 0.1319006549130226, Training accuracy: 96.29014396456257%, Validation loss: 1.8295849561691284, Validation accuracy: 73.84615384615384%
Epoch: 6, Training loss: 0.0981348352185611, Training accuracy: 97.84053156146179%, Validation loss: 1.3715955913066864, Validation accuracy: 76.41025641025641%
Epoch: 7, Training loss: 0.098916076

In [195]:
model = get_model_VGG()
trained_model = train_model(model,epochs=3)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.3815136654623623, Training accuracy: 47.3421926910299%, Validation loss: 1.09053435921669, Validation accuracy: 66.15384615384616%
Epoch: 2, Training loss: 0.9501745680282856, Training accuracy: 65.39313399778516%, Validation loss: 1.0638394206762314, Validation accuracy: 68.2051282051282%
Epoch: 3, Training loss: 0.7075542889792343, Training accuracy: 73.20044296788483%, Validation loss: 1.0860777795314789, Validation accuracy: 67.17948717948718%
Finished Training
Accuracy of the model on the test images: 65.12820512820512%
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.38      0.47        24
           1       0.60      0.89      0.71        73
           2       0.88      0.78      0.82        27
           3       0.50      0.13      0.21        15
           4       0.62      0.29      0.40        17
           5       0.69      0.64      0.67        39

    accuracy                           

In [196]:
model = get_model_VGG()
trained_model = train_model(model,epochs=5)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.3521068692207336, Training accuracy: 46.12403100775194%, Validation loss: 1.235445350408554, Validation accuracy: 65.64102564102564%
Epoch: 2, Training loss: 0.9256667392007236, Training accuracy: 65.17165005537099%, Validation loss: 1.1238585412502289, Validation accuracy: 65.64102564102564%
Epoch: 3, Training loss: 0.6955751211478792, Training accuracy: 73.75415282392026%, Validation loss: 1.1905182003974915, Validation accuracy: 66.66666666666667%
Epoch: 4, Training loss: 0.5781327640188152, Training accuracy: 78.7375415282392%, Validation loss: 1.4986803233623505, Validation accuracy: 66.66666666666667%
Epoch: 5, Training loss: 0.40663032798931514, Training accuracy: 84.99446290143965%, Validation loss: 1.3009531795978546, Validation accuracy: 71.28205128205128%
Finished Training
Accuracy of the model on the test images: 66.15384615384616%
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.62      

In [197]:
model = get_model_VGG()
trained_model = train_model(model,epochs=10)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.3699009007421032, Training accuracy: 46.56699889258029%, Validation loss: 1.0508307963609695, Validation accuracy: 64.1025641025641%
Epoch: 2, Training loss: 1.0562508517298206, Training accuracy: 61.24031007751938%, Validation loss: 1.2050221264362335, Validation accuracy: 66.15384615384616%
Epoch: 3, Training loss: 0.7741676281238424, Training accuracy: 70.65337763012181%, Validation loss: 1.016515091061592, Validation accuracy: 71.7948717948718%
Epoch: 4, Training loss: 0.6453408424196572, Training accuracy: 76.35658914728683%, Validation loss: 1.0590747743844986, Validation accuracy: 69.74358974358974%
Epoch: 5, Training loss: 0.5268838837228972, Training accuracy: 80.0110741971207%, Validation loss: 1.1854523569345474, Validation accuracy: 67.6923076923077%
Epoch: 6, Training loss: 0.41363473493477393, Training accuracy: 85.60354374307863%, Validation loss: 1.4340049028396606, Validation accuracy: 70.25641025641026%
Epoch: 7, Training loss: 0.31422587920

In [207]:
model = get_model_resnet()
trained_model = train_model(model,epochs=3)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 0.9598960465398328, Training accuracy: 64.72868217054264%, Validation loss: 1.4220524728298187, Validation accuracy: 62.56410256410256%
Epoch: 2, Training loss: 0.3688853645119174, Training accuracy: 87.87375415282392%, Validation loss: 1.342769831418991, Validation accuracy: 67.6923076923077%
Epoch: 3, Training loss: 0.21050604689737845, Training accuracy: 92.80177187153932%, Validation loss: 0.9091637581586838, Validation accuracy: 71.7948717948718%
Finished Training
Accuracy of the model on the test images: 70.25641025641026%
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.54      0.57        24
           1       0.72      0.86      0.78        73
           2       0.86      0.89      0.87        27
           3       0.44      0.27      0.33        15
           4       0.53      0.53      0.53        17
           5       0.77      0.62      0.69        39

    accuracy                         

In [210]:
model = get_model_resnet()
trained_model = train_model(model,epochs=5)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 0.9836982057012361, Training accuracy: 64.45182724252491%, Validation loss: 1.0147064328193665, Validation accuracy: 67.6923076923077%
Epoch: 2, Training loss: 0.34074186353847896, Training accuracy: 87.65227021040974%, Validation loss: 1.2127640694379807, Validation accuracy: 67.17948717948718%
Epoch: 3, Training loss: 0.17635074990062877, Training accuracy: 94.18604651162791%, Validation loss: 0.870762050151825, Validation accuracy: 74.87179487179488%
Epoch: 4, Training loss: 0.14890534058213234, Training accuracy: 95.29346622369879%, Validation loss: 0.9477849826216698, Validation accuracy: 71.28205128205128%
Epoch: 5, Training loss: 0.13490611250544415, Training accuracy: 97.00996677740864%, Validation loss: 1.5366940796375275, Validation accuracy: 69.23076923076923%
Finished Training
Accuracy of the model on the test images: 67.6923076923077%
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.62    

In [209]:
model = get_model_resnet()
trained_model = train_model(model,epochs=10)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.0154001610032444, Training accuracy: 62.90143964562569%, Validation loss: 1.1974450796842575, Validation accuracy: 72.3076923076923%
Epoch: 2, Training loss: 0.36031386821434414, Training accuracy: 87.59689922480621%, Validation loss: 1.4713086783885956, Validation accuracy: 65.64102564102564%
Epoch: 3, Training loss: 0.1650459753542111, Training accuracy: 94.57364341085271%, Validation loss: 1.1751352399587631, Validation accuracy: 70.25641025641026%
Epoch: 4, Training loss: 0.15269257642071823, Training accuracy: 95.40420819490586%, Validation loss: 1.6543059349060059, Validation accuracy: 67.6923076923077%
Epoch: 5, Training loss: 0.10120271974853401, Training accuracy: 96.78848283499447%, Validation loss: 1.696806013584137, Validation accuracy: 71.28205128205128%
Epoch: 6, Training loss: 0.06298565864562988, Training accuracy: 98.56035437430786%, Validation loss: 1.6771423518657684, Validation accuracy: 68.2051282051282%
Epoch: 7, Training loss: 0.0379708

In [113]:
model = get_model_resnet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.1851976352982816, Training accuracy: 57.308970099667775%, Validation loss: 1.3689427421643183, Validation accuracy: 57.94871794871795%
Epoch: 2, Training loss: 0.680096819073753, Training accuracy: 75.91362126245848%, Validation loss: 0.8659154222561762, Validation accuracy: 70.76923076923077%
Epoch: 3, Training loss: 0.4324862684032558, Training accuracy: 84.9390919158361%, Validation loss: 1.261734196772942, Validation accuracy: 63.58974358974359%
Epoch: 4, Training loss: 0.28072028928559967, Training accuracy: 90.2547065337763%, Validation loss: 1.0815186248375819, Validation accuracy: 68.71794871794872%
Epoch: 5, Training loss: 0.2183475135372276, Training accuracy: 92.52491694352159%, Validation loss: 1.4864197786037738, Validation accuracy: 66.66666666666667%
Epoch: 6, Training loss: 0.20954040523651427, Training accuracy: 93.02325581395348%, Validation loss: 1.176123123902541, Validation accuracy: 67.17948717948718%
Epoch: 7, Training loss: 0.141680413

In [187]:
model = get_model_alexnet()
trained_model = train_model(model,epochs=3)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.2678824416522323, Training accuracy: 50.332225913621265%, Validation loss: 1.2290533781051636, Validation accuracy: 65.12820512820512%
Epoch: 2, Training loss: 0.765293346396808, Training accuracy: 69.82281284606866%, Validation loss: 1.1828121989965439, Validation accuracy: 67.6923076923077%
Epoch: 3, Training loss: 0.4740357244836873, Training accuracy: 82.11517165005537%, Validation loss: 1.7304570227861404, Validation accuracy: 68.2051282051282%
Finished Training
Accuracy of the model on the test images: 64.61538461538461%
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.42      0.47        24
           1       0.70      0.75      0.72        73
           2       0.84      0.78      0.81        27
           3       1.00      0.13      0.24        15
           4       0.38      0.35      0.36        17
           5       0.59      0.82      0.69        39

    accuracy                         

In [190]:
model = get_model_alexnet()
trained_model = train_model(model,epochs=5)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.2920500656654095, Training accuracy: 48.72646733111849%, Validation loss: 1.5529797673225403, Validation accuracy: 59.48717948717949%
Epoch: 2, Training loss: 0.7555756805271938, Training accuracy: 71.70542635658914%, Validation loss: 1.399549663066864, Validation accuracy: 64.1025641025641%
Epoch: 3, Training loss: 0.4775946880209035, Training accuracy: 82.61351052048727%, Validation loss: 1.4125100374221802, Validation accuracy: 69.23076923076923%
Epoch: 4, Training loss: 0.32399496744418965, Training accuracy: 88.53820598006645%, Validation loss: 2.0065242797136307, Validation accuracy: 66.66666666666667%
Epoch: 5, Training loss: 0.2112150063802456, Training accuracy: 92.35880398671097%, Validation loss: 1.647813305258751, Validation accuracy: 64.1025641025641%
Finished Training
Accuracy of the model on the test images: 66.66666666666667%
Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.54      0.

In [158]:
model = get_model_alexnet()
trained_model = train_model(model,epochs=10)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.1908258367003055, Training accuracy: 53.43300110741971%, Validation loss: 1.0598765356200082, Validation accuracy: 64.61538461538461%
Epoch: 2, Training loss: 0.7053823952089276, Training accuracy: 72.70210409745293%, Validation loss: 1.1744089722633362, Validation accuracy: 65.12820512820512%
Epoch: 3, Training loss: 0.4710814701883416, Training accuracy: 83.38870431893687%, Validation loss: 1.0910744667053223, Validation accuracy: 64.61538461538461%
Epoch: 4, Training loss: 0.27338524554905136, Training accuracy: 90.14396456256921%, Validation loss: 1.2737454686846053, Validation accuracy: 65.12820512820512%
Epoch: 5, Training loss: 0.19011479308992102, Training accuracy: 93.85382059800665%, Validation loss: 1.2779672145843506, Validation accuracy: 66.15384615384616%
Epoch: 6, Training loss: 0.17239771443500854, Training accuracy: 93.96456256921373%, Validation loss: 1.2948667321886336, Validation accuracy: 67.6923076923077%
Epoch: 7, Training loss: 0.13009

In [159]:
model = get_model_alexnet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 1.2179580529530842, Training accuracy: 53.98671096345515%, Validation loss: 1.0619439312389918, Validation accuracy: 65.64102564102564%
Epoch: 2, Training loss: 0.6933513932060777, Training accuracy: 74.08637873754152%, Validation loss: 1.0820266689573015, Validation accuracy: 68.2051282051282%
Epoch: 3, Training loss: 0.46243443536131007, Training accuracy: 82.89036544850498%, Validation loss: 1.0185822333608354, Validation accuracy: 67.17948717948718%
Epoch: 4, Training loss: 0.31267687077062173, Training accuracy: 88.70431893687707%, Validation loss: 1.2470426048551286, Validation accuracy: 63.58974358974359%
Epoch: 5, Training loss: 0.19245363902627377, Training accuracy: 93.63233665559247%, Validation loss: 1.388728107724871, Validation accuracy: 62.56410256410256%
Epoch: 6, Training loss: 0.15043253039843157, Training accuracy: 95.01661129568106%, Validation loss: 1.6416954398155212, Validation accuracy: 63.07692307692308%
Epoch: 7, Training loss: 0.10527

In [213]:
model = get_model_densenet()
trained_model = train_model(model,epochs=3)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 89.6MB/s]


Epoch: 1, Training loss: 0.8247645147915544, Training accuracy: 69.82281284606866%, Validation loss: 1.059909924864769, Validation accuracy: 69.23076923076923%
Epoch: 2, Training loss: 0.211917189174685, Training accuracy: 93.57696566998892%, Validation loss: 0.976599931716919, Validation accuracy: 71.28205128205128%
Epoch: 3, Training loss: 0.06775839273528807, Training accuracy: 98.61572535991141%, Validation loss: 1.1788903623819351, Validation accuracy: 74.35897435897436%
Finished Training
Accuracy of the model on the test images: 71.7948717948718%
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.46      0.54        24
           1       0.71      0.85      0.77        73
           2       0.84      0.78      0.81        27
           3       0.78      0.47      0.58        15
           4       0.44      0.65      0.52        17
           5       0.88      0.72      0.79        39

    accuracy                          

In [221]:
model = get_model_densenet()
trained_model = train_model(model,epochs=5)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 0.8590661420904356, Training accuracy: 66.38981173864894%, Validation loss: 1.090654969215393, Validation accuracy: 67.6923076923077%
Epoch: 2, Training loss: 0.23065630706219836, Training accuracy: 93.30011074197121%, Validation loss: 0.9822745621204376, Validation accuracy: 71.28205128205128%
Epoch: 3, Training loss: 0.07158285482176419, Training accuracy: 98.33887043189368%, Validation loss: 0.9645752161741257, Validation accuracy: 72.82051282051282%
Epoch: 4, Training loss: 0.025394082069396973, Training accuracy: 99.6124031007752%, Validation loss: 1.0340644717216492, Validation accuracy: 70.25641025641026%
Epoch: 5, Training loss: 0.02523759370348577, Training accuracy: 99.6124031007752%, Validation loss: 0.8726556375622749, Validation accuracy: 68.71794871794872%
Finished Training
Accuracy of the model on the test images: 71.7948717948718%
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.50     

In [222]:
model = get_model_densenet()
trained_model = train_model(model,epochs=10)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 0.8413676619529724, Training accuracy: 68.10631229235881%, Validation loss: 1.2075041234493256, Validation accuracy: 67.17948717948718%
Epoch: 2, Training loss: 0.20588201078875312, Training accuracy: 94.57364341085271%, Validation loss: 0.786307580769062, Validation accuracy: 72.82051282051282%
Epoch: 3, Training loss: 0.0742627190875596, Training accuracy: 98.2281284606866%, Validation loss: 0.995068147778511, Validation accuracy: 67.17948717948718%
Epoch: 4, Training loss: 0.02699102090414742, Training accuracy: 99.44629014396456%, Validation loss: 0.9616588354110718, Validation accuracy: 73.33333333333333%
Epoch: 5, Training loss: 0.015328869063021808, Training accuracy: 99.83388704318936%, Validation loss: 0.8615427501499653, Validation accuracy: 72.3076923076923%
Epoch: 6, Training loss: 0.014326027458019811, Training accuracy: 99.6124031007752%, Validation loss: 1.2391501367092133, Validation accuracy: 68.71794871794872%
Epoch: 7, Training loss: 0.018628

In [252]:
model = get_model_densenet()
trained_model = train_model(model,epochs=20)
evaluate_classifier(test_dataloader, trained_model, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Training loss: 0.8734383624175499, Training accuracy: 66.38981173864894%, Validation loss: 1.2653553783893585, Validation accuracy: 68.2051282051282%
Epoch: 2, Training loss: 0.2656413010995963, Training accuracy: 92.24806201550388%, Validation loss: 1.11929751932621, Validation accuracy: 72.3076923076923%
Epoch: 3, Training loss: 0.10597261715808819, Training accuracy: 97.28682170542636%, Validation loss: 1.0208176374435425, Validation accuracy: 74.35897435897436%
Epoch: 4, Training loss: 0.046826563004789684, Training accuracy: 99.16943521594685%, Validation loss: 1.26620202511549, Validation accuracy: 74.35897435897436%
Epoch: 5, Training loss: 0.03552374853913126, Training accuracy: 99.16943521594685%, Validation loss: 1.0763324946165085, Validation accuracy: 71.7948717948718%
Epoch: 6, Training loss: 0.01930790259663401, Training accuracy: 99.6124031007752%, Validation loss: 1.3661493062973022, Validation accuracy: 73.84615384615384%
Epoch: 7, Training loss: 0.0115702782

In [ ]:
labels_dict = {"crimp": 0, "pinch": 1, "pocket": 2, "jug": 3, "edge": 4, "sloper": 5}
